**Data from: https://ai-jobs.net/salaries/**\
**Date retrieved: 20 June 2024**

In [32]:
import pandas as pd
import country_converter as coco

In [33]:
df = pd.read_csv('raw.csv')

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22144 entries, 0 to 22143
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           22144 non-null  int64 
 1   experience_level    22144 non-null  object
 2   employment_type     22144 non-null  object
 3   job_title           22144 non-null  object
 4   salary              22144 non-null  int64 
 5   salary_currency     22144 non-null  object
 6   salary_in_usd       22144 non-null  int64 
 7   employee_residence  22144 non-null  object
 8   remote_ratio        22144 non-null  int64 
 9   company_location    22144 non-null  object
 10  company_size        22144 non-null  object
dtypes: int64(4), object(7)
memory usage: 1.9+ MB


In [35]:
# Initial shape of df
df.shape

(22144, 11)

In [36]:
df.isnull().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [37]:
df['job_title'].value_counts()

job_title
Data Scientist                     5166
Data Engineer                      4503
Data Analyst                       3129
Machine Learning Engineer          2403
Research Scientist                  957
                                   ... 
Staff Machine Learning Engineer       1
AWS Data Architect                    1
Analytics Engineering Manager         1
Lead AI Engineer                      1
Marketing Data Scientist              1
Name: count, Length: 160, dtype: int64

In [38]:
df.shape

(22144, 11)

In [39]:
# Filter df to targeted fields only
selected_field = ['Data Scientist', 'Data Engineer', 'Data Analyst','Machine Learning Engineer','AI Engineer','Business Intelligence Analyst']
df = df[df['job_title'].isin(selected_field)]
df.shape

(15702, 11)

In [40]:
df.groupby('job_title').size().reset_index(name='count')

job_title  count
0                    AI Engineer    196
1  Business Intelligence Analyst    305
2                   Data Analyst   3129
3                  Data Engineer   4503
4                 Data Scientist   5166
5      Machine Learning Engineer   2403

In [41]:
df = df.rename(columns={
    'work_year': 'year',
    'job_title': 'career',
    'remote_ratio': 'remote_status'
})

In [42]:
df['experience_level'] = df['experience_level'].replace({'EN': 'Entry-level', 'MI': 'Intermediate','SE':'Senior','EX':'Director'})
df['employment_type'] = df['employment_type'].replace({'PT': 'Part-time', 'FT': 'Full-time','CT':'Contract','FL':'Freelance'})
df['company_size'] = df['company_size'].replace({'S': 'Small (<50 employees)', 'M': 'Medium (50-250 employees)','L':'Large (>250 employees)'})

In [43]:
df

year experience_level employment_type          career   salary  \
2      2024     Intermediate       Full-time   Data Engineer   196900   
3      2024     Intermediate       Full-time   Data Engineer   175457   
4      2024           Senior       Full-time   Data Engineer   235400   
5      2024           Senior       Full-time   Data Engineer   209720   
8      2024     Intermediate       Full-time  Data Scientist   196900   
...     ...              ...             ...             ...      ...   
22135  2021     Intermediate       Full-time  Data Scientist   160000   
22137  2021     Intermediate       Full-time   Data Engineer    24000   
22139  2020           Senior       Full-time  Data Scientist   412000   
22141  2020      Entry-level       Full-time  Data Scientist   105000   
22143  2021           Senior       Full-time  Data Scientist  7000000   

      salary_currency  salary_in_usd employee_residence  remote_status  \
2                 USD         196900                 US              0   
3                 USD         175457                 US              0   
4                 USD         235400                 US              0   
5                 USD         209720                 US              0   
8                 USD         196900                 US              0   
...               ...            ...                ...            ...   
22135             SGD         119059                 SG            100   
22137             EUR          28369                 MT             50   
22139             USD         412000                 US            100   
22141             USD         105000                 US            100   
22143             INR          94665                 IN             50   

      company_location               company_size  
2                   US  Medium (50-250 employees)  
3                   US  Medium (50-250 employees)  
4                   US  Medium (50-250 employees)  
5                   US  Medium (50-250 employees)  
8                   US  Medium (50-250 employees)  
...                ...                        ...  
22135               IL  Medium (50-250 employees)  
22137               MT     Large (>250 employees)  
22139               US     Large (>250 employees)  
22141               US      Small (<50 employees)  
22143               IN     Large (>250 employees)  

[15702 rows x 11 columns]

In [44]:
df.to_csv('clean.csv', index = False)

In [45]:
df = pd.read_csv('clean.csv')
dff = df.groupby('company_location').size().reset_index(name='count')
total_count = dff['count'].sum()
dff['percentage'] = (dff['count'] / total_count) * 100

def alpha2_to_alpha3(alpha2):
    try:
        return coco.convert(names=alpha2, to='ISO3')
    except coco.CountryNotFoundException:
        return None
 
def alpha2_to_country_name(alpha2):
    try:
        return coco.convert(names=alpha2, to='name_short')
    except coco.CountryNotFoundException:
        return None

dff['country_name'] = dff['company_location'].apply(alpha2_to_country_name)
dff['company_location'] = dff['company_location'].apply(alpha2_to_alpha3)

In [46]:
dff

company_location  count  percentage          country_name
0               AND      1    0.006369               Andorra
1               ARE      3    0.019106  United Arab Emirates
2               ARM      1    0.006369               Armenia
3               ARG     13    0.082792             Argentina
4               AUT      9    0.057318               Austria
..              ...    ...         ...                   ...
58              TUR      9    0.057318               Türkiye
59              UKR      8    0.050949               Ukraine
60              USA  13997   89.141511         United States
61              VNM      4    0.025474               Vietnam
62              ZAF     10    0.063686          South Africa

[63 rows x 4 columns]

In [47]:
dff.to_csv('mapdata.csv',index = False)
